# Snow Sensor Usability Masks
First look at classified areas for Passive Microwave Sensor Usability for observing snow properties.
Here, we compare input constraint layers and classified images. 

**Victoria Ly** (University of Washington)

***
#### Google Earth Engine: 
- [Google Earth Engine NDSI Masks](https://code.earthengine.google.com/1c8fdf62b558feed5bb84c683dd30f05)
- [Google Earth Engine Classification Masks](https://code.earthengine.google.com/1acc3ce95c22a918bbda912b94a108b6)

### Set up Google Earth Engine API

In [1]:
# Import libraries.
import ee

from IPython.display import Image
import matplotlib.pyplot as plt
import rasterio
import xarray as xr
import numpy as np

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/wwEIdKjNV8-3JGMVL7CMHxAWvyjIdI0hDMTvfP4f_4MrUVW1iQUpJjw

Successfully saved authorization token.


### PM sensor Classifications
Previously, I classified areas where Passive Microwave would expect to work. Given the following constraints:
- LST < 0 degrees C
- Tree Canopy Cover = 20-30%
- SWE = 2-150mm

In [3]:
# Import an exported asset 
img = ee.Image('users/victorialy/classification_month01')

In [4]:
print(img.getInfo())

{'type': 'Image', 'bands': [{'id': 'band_classified', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [1551, 290], 'crs': 'EPSG:4326', 'crs_transform': [0.22457882102988036, 0, -168.2095369513804, 0, -0.22457882102988036, 85.11537317032466]}], 'version': 1581708812866376, 'id': 'users/victorialy/classification_month01', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-37.560807821939015, 19.709680030592384], [-18.524243683211537, 19.565324647636942], [8.670847891494898, 19.153561091870507], [33.14643035868383, 19.70967999639198], [52.182994525167125, 19.56532464955654], [73.93906776700103, 19.56532469808809], [92.97563190966589, 19.709680032870622], [112.0121960001795, 19.565324646789644], [139.20728760949774, 19.15356107059357], [166.40237920766884, 19.565324622095375], [-179.7614692725366, 19.87102099849134], [-170.7718422403738, 85.18313433976225], [-168.3337333083568, 19.361844038339775], [-146.34117428739177, 1

In [5]:
#Define a region of interest
roi = ee.Geometry.Polygon([[-168, 85],
                            [-168, 20],
                            [180, 20],
                            [180, 85]])

In [6]:
#Take a quick glance
Image(url=img.getThumbUrl({'bands':['band_classified'], 
                           'min': 0, 'max': 2, 'palette': ["c15433","1d8e30","dcd72b"], 
                           'region': ee.Geometry.Polygon([[-168, 85],
                            [-168, 20],
                            [180, 20],
                            [180, 85]]),
                          }));

In [7]:
#Take a quick glance
Image(url=img.getThumbUrl({'bands':['band_classified'], 
                           'min': 0, 'max': 2, 'palette': ["c15433","1d8e30","dcd72b"]}))

In [8]:
#Add lat long
imgll = img.addBands(ee.Image.pixelLonLat())
print(imgll.getInfo())

{'type': 'Image', 'bands': [{'id': 'band_classified', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [1551, 290], 'crs': 'EPSG:4326', 'crs_transform': [0.22457882102988036, 0, -168.2095369513804, 0, -0.22457882102988036, 85.11537317032466]}, {'id': 'longitude', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'latitude', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'version': 1581708812866376, 'id': 'users/victorialy/classification_month01', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-37.560807821939015, 19.709680030592384], [-18.524243683211537, 19.565324647636942], [8.670847891494898, 19.153561091870507], [33.14643035868383, 19.70967999639198], [52.182994525167125, 19.56532464955654], [73.93906776700103, 19.56532469808809], [92.97563190966589, 19.709680032870622]

In [9]:
#Convert lat and long to arrays
data = np.array(imgll.get('band_classified').getInfo())
lats = np.array((ee.Array(imgll.get("latitude")).getInfo()))
lons = np.array((ee.Array(imgll.get("longitude")).getInfo()))

EEException: Array: Parameter 'values' is required.

In [10]:
# Plot January Snow Presence/Absence, where NDSI > 0.4
img2 = ee.Image('users/victorialy/NDSI_gt40_month01')
Image(url=img2.getThumbUrl({'bands':['NDSI_gt40'], 
                           'min': 0, 'max': 1, 'palette': ["000000","27e1ff"]}))

In [ ]:
# Plot tree canopy cover
img3 = ee.Image('users/victorialy/tree_25km_mean')
Image(url=img3.getThumbUrl({'bands':['treecover2000'], 
                           'min': 0, 'max': 100, 'palette': ['000000', 'afce56', '5f9c00', '0e6a00', '003800']}))

### Import Classified Image from GEE

Previously, I classified areas where Passive Microwave would expect to work. Given the following constraints:
- LST < 0 degrees C
- Tree Canopy Cover = 20-30%
- SWE = 2-150mm

In [11]:
# Import classified image, with all input bands
img4 = ee.Image('users/victorialy/test_classification_month01')
Image(url=img4.getThumbUrl({'bands':['Sensor_Classification'], 
                           'min': 0, 'max': 2, 'palette': ["c15433","1d8e30","dcd72b"]}))

In [12]:
# Plot tree canopy cover
Image(url=img4.getThumbUrl({'bands':['TCC_avg'], 
                           'min': 0, 'max': 100, 'palette': ['000000', 'afce56', '5f9c00', '0e6a00', '003800']}))

In [14]:
# Plot SWE
Image(url=img4.getThumbUrl({'bands':['SWE_avg'], 
                           "max":200,"palette":["000000","7ce8ed","21edbe","0c35ff"]}))

In [18]:
# Plot LST
Image(url=img4.getThumbUrl({'bands':['LST_avg'], 
                           "min":-8,"max":0,"palette":["4286f4","000b5e","96048f"]}))